# Install OR-Tools (For Colab Only)

In [3]:
!pip install ortools

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\Scripts\pip-script.py", line 10, in <module>
    sys.exit(main())
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_internal\cli\main.py", line 75, in main
    return command.main(cmd_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 114, in main
    return self._main(args)
  File "C:\ProgramData\Anaconda3\lib\contextlib.py", line 120, in __exit__
    next(self.gen)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_internal\cli\command_context.py", line 28, in main_context
    yield
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\contextlib2.py", line 479, in __exit__
    _reraise_with_existing_context(exc_details)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\contextlib2.py", line 340, in _reraise_with_existing_context
    raise exc_details[1]
  File "C:\ProgramData\Anaconda3\lib\contextlib.py", line 131, in __exit__
    self.g

## Import necessary libraries

In [4]:
from ortools.linear_solver import pywraplp
import random

# Problem from 1995 MBA exam

A project manager in a company is considering a portfolio of 10 large project investments. These investments differ in the estimated long-run profit (net present value) they will generate as well as in the amount of capital required.

Let $P_j$ and $C_j$ denote the estimated profit and capital required (both given in units of millions of £) for investment opportunity j (j=1,...,10) respectively. The total amount of capital available for these investments is Q (in units of millions of £)

Investment opportunities 3 and 4 are mutually exclusive and so are 5 and 6. Furthermore, neither 5 nor 6 can be undertaken unless either 3 or 4 is undertaken. At least two and at most four investment opportunities have to be undertaken from the set {1,2,7,8,9,10}.

The project manager wishes to select the combination of capital investments that will maximise the total estimated long-run profit subject to the restrictions described above.

Formulate this problem using an integer programming model and comment on the difficulties of solving this model. (Do not actually solve it).

What are the advantages and disadvantages of using this model for portfolio selection?

## First let's setup the problem

The problem is defined without real values for profits, capital requirements, and capital so we will have to make these values up in order to have a solvable problem.

In [5]:
num_investments = 10
profit = [random.randint(1,100) for _ in range(num_investments)]
cost = [random.randint(1,50) for _ in range(num_investments)]
capital_available = 127

## Now let's create the IP model

What we mean by this, is we need to define our variables, objective, and constraints like we do when we write down our model on paper (or white board).  Only additional step we need to take when coding our model is creating the solver object with the correct type of solver for our problem.  In this case we use SCIP because we are solving an integer programming problem.

In [6]:
solver = pywraplp.Solver.CreateSolver('SCIP')

variables = [solver.IntVar(0, 1, str(i)) for i in range(num_investments)]

# objective
solver.Maximize(sum([variables[i] * (profit[i] - cost[i]) for i in range(num_investments)]))

# constraints
solver.Add(sum([variables[i] * cost[i] for i in range(num_investments)]) <= capital_available) # capital constraint

solver.Add(variables[2] + variables[3] <= 1) # exclusivity of 3 and 4
solver.Add(variables[4] + variables[5] <= 1) # exclusifity of 5 and 6

solver.Add(variables[4] + variables[5] <= variables[2] + variables[3]) # 5 and 6 depend on 3 and 4
solver.Add(sum([variables[i] for i in range(num_investments) if i in [0,1,6,7,8,9]]) >= 2)
solver.Add(sum([variables[i] for i in range(num_investments) if i in [0,1,6,7,8,9]]) <= 4)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x0000026F78FCC120> >

## Now let's solve the model

We need to tell the solver to solve the model and record the result.  If we forget to look at the result of the solver, we will not know if the answer it provides is an actual solution or the best solution.  Therefore we need to check the result it provides before doing anything with the solution.

In [7]:
status = solver.Solve()

if status == solver.OPTIMAL:
  print(f'Profits:              {profit}')
  print(f'Costs:                {cost}')
  print(f'Selected Investments: {[v.solution_value() for v in variables]}')
else:
  print('infeasible')

Profits:              [31, 2, 70, 58, 21, 48, 36, 25, 48, 12]
Costs:                [2, 27, 10, 45, 18, 2, 29, 21, 11, 50]
Selected Investments: [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0]


# 1994 MBA Exam

A food is manufactured by refining raw oils and blending them together. The raw oils come in two categories:

Vegetable oil:
* VEG1
* VEG2

Non-vegetable oil:
* OIL1
* OIL2
* OIL3

The prices for buying each oil are given below (in £/tonne)

|VEG1|VEG2| OIL1 | OIL2 | OIL3 |
|---|---|---|---|---|
|115 | 128 | 132 | 109 | 114 |

The final product sells at £180 per tonne. Vegetable oils and non-vegetable oils require different production lines for refining. It is not possible to refine more than 210 tonnes of vegetable oils and more than 260 tonnes of non-vegetable oils. There is no loss of weight in the refining process and the cost of refining may be ignored.

There is a technical restriction relating to the hardness of the final product. In the units in which hardness is measured this must lie between 3.5 and 6.2. It is assumed that hardness blends linearly and that the hardness of the raw oils is:

|VEG1 | VEG2 | OIL1 | OIL2 | OIL3 |
|---|---|---|---|---|
|8.8 | 6.2  | 1.9 |  4.3 |  5.1 |

It is required to determine what to buy and how to blend the raw oils so that the company maximises its profit.

1. Formulate the above problem as a linear program. (Do not actually solve it).
2. What assumptions do you make in solving this problem by linear programming?

The following extra conditions are imposed on the food manufacture problem stated above as a result of the production process involved:

* the food may never be made up of more than 3 raw oils
* if an oil (vegetable or non-vegetable) is used, at least 30 tonnes of that oil must be used
* if either of VEG1 or VEG2 are used then OIL2 must also be used

Introducing 0-1 integer variables extend the linear programming model you have developed to encompass these new extra conditions.

# Big M Example

In the planning of the monthly production for the next six months a company must, in each month, operate either a normal shift or an extended shift (if it produces at all). A normal shift costs £100,000 per month and can produce up to 5,000 units per month. An extended shift costs £180,000 per month and can produce up to 7,500 units per month. Note here that, for either type of shift, the cost incurred is fixed by a union guarantee agreement and so is independent of the amount produced.

It is estimated that changing from a normal shift in one month to an extended shift in the next month costs an extra £15,000. No extra cost is incurred in changing from an extended shift in one month to a normal shift in the next month.

The cost of holding stock is estimated to be £2 per unit per month (based on the stock held at the end of each month) and the initial stock is 3,000 units (produced by a normal shift). The amount in stock at the end of month 6 should be at least 2,000 units. The demand for the company's product in each of the next six months is estimated to be as shown below:

|Month |  1 |    2  |   3  |   4   |  5  |   6 |
|---|---|---|---|---|---|---|
|Demand | 6,000 | 6,500 | 7,500 | 7,000 | 6,000 | 6,000 |
 
Production constraints are such that if the company produces anything in a particular month it must produce at least 2,000 units. If the company wants a production plan for the next six months that avoids stockouts, formulate their problem as an integer program.